# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense, Masking, LSTM, Embedding

![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [2]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [3]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [4]:
sentences = tf.strings.split(data_examples, sep='\t').to_tensor().numpy()

In [5]:
eng_sentences = sentences[:,0]
ger_sentences = sentences[:,1]

In [6]:
eng_sentences = [preprocess_sentence(sentence.decode('utf-8')) for sentence in eng_sentences]
ger_sentences = ['<start>' + preprocess_sentence(sentence.decode('utf-8')) + '<end>' for sentence in ger_sentences]

In [7]:
additional_filters = '—’‘“”'
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' + additional_filters,
                      char_level=False,
                      oov_token='<UNK>')

tokenizer.fit_on_texts(ger_sentences)

tokenizer_config = tokenizer.get_config()
num_unique_german_tokens = max(tokenizer.word_index.values()) + 1

In [8]:
import json
index_word = json.loads(tokenizer_config['index_word'])
word_index = json.loads(tokenizer_config['word_index'])

In [9]:
import numpy as np
import random
random_indices = tf.random.uniform((1,5), minval=0, dtype=tf.int32, maxval=len(eng_sentences))
random_ger_sentences = np.array(ger_sentences)[random_indices]
random_eng_sentences = np.array(eng_sentences)[random_indices]
tokenized_ger_sentences = tokenizer.texts_to_sequences(ger_sentences)
random_tokenized_ger_sentences = np.array(tokenized_ger_sentences)[random_indices]

In [10]:
for a,b,c in zip(random_eng_sentences, random_ger_sentences, random_tokenized_ger_sentences):
    print(a,'\t\t',b,'\t\t', c)

it's bad for you . 		 <start>es ist schlecht fuer dich .<end> 		 [2, 8, 6, 254, 74, 25, 3]
don't rush me . 		 <start>hetz mich nicht .<end> 		 [2, 3543, 20, 10, 3]
mary shrieked . 		 <start>maria kreischte .<end> 		 [2, 116, 3651, 3]
i'll be fine . 		 <start>ich komme schon klar .<end> 		 [2, 4, 177, 93, 510, 3]
can i hug you ? 		 <start>darf ich sie umarmen ?<end> 		 [2, 131, 4, 7, 1447, 3]


In [11]:
padded_ger_sentences = tf.keras.preprocessing.sequence.pad_sequences(tokenized_ger_sentences, padding='post')

## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [12]:
embedding_layer = tf.keras.models.load_model('./models/tf2-preview_nnlm-en-dim128_1')

In [13]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [14]:
from sklearn.model_selection import train_test_split
eng_train, eng_test = train_test_split(eng_sentences, test_size=0.2)
ger_train, ger_test = train_test_split(padded_ger_sentences, test_size=0.2)

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((eng_train, ger_train))
test_dataset = tf.data.Dataset.from_tensor_slices((eng_test, ger_test))

In [16]:
def split_at_spaces(eng_sentence, ger_sentence):
    return tf.strings.split(eng_sentence, sep=' '), ger_sentence   

In [17]:
def embbed(eng_sentence, ger_sentence):
    return embedding_layer(eng_sentence), ger_sentence

In [18]:
def filter_longer(eng_sentence, ger_sentence):
    return tf.shape(eng_sentence)[0] <= 13

In [19]:
def pad_sentence(eng_sentence, ger_sentence):
    sentence_length = tf.shape(eng_sentence)[0]
    MAX_LENGTH = 13
    paddings =  [[MAX_LENGTH - sentence_length, 0], [0,0]]
    eng_sentence_padded = tf.pad(eng_sentence, paddings)
    return eng_sentence_padded, ger_sentence

In [20]:
train_dataset = train_dataset.map(split_at_spaces).map(embbed).filter(filter_longer).map(pad_sentence)
test_dataset = test_dataset.map(split_at_spaces).map(embbed).filter(filter_longer).map(pad_sentence)

In [21]:
train_dataset = train_dataset.batch(16, drop_remainder=True)
test_dataset = test_dataset.batch(16, drop_remainder=True)

In [22]:
print(train_dataset.element_spec)
print(test_dataset.element_spec)

(TensorSpec(shape=(16, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 12), dtype=tf.int32, name=None))
(TensorSpec(shape=(16, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 12), dtype=tf.int32, name=None))


In [23]:
for x,y in train_dataset.take(1):
    print(tf.shape(x))
    print(tf.shape(y))

tf.Tensor([ 16  13 128], shape=(3,), dtype=int32)
tf.Tensor([16 12], shape=(2,), dtype=int32)


## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [24]:
class EmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        super(EmbeddingLayer, self).__init__(**kwargs)
        self.v = tf.Variable(initial_value=tf.random.uniform((1,1,128)) ,shape=(1,1,128))
        
    def call(self, inputs):   
        v = tf.tile(self.v, (tf.shape(inputs)[0],1,1))      
        return tf.concat([inputs, v ], axis=1)
        

In [25]:
el = EmbeddingLayer()

In [26]:
for eng_sentence, ger_sentence in train_dataset.take(1):
    x = el(eng_sentence)
    print(tf.shape(x))

tf.Tensor([ 16  14 128], shape=(3,), dtype=int32)


## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [27]:
class Encoder(Model):
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding_layer = el
        self.masking_layer = Masking(mask_value=0)
        self.lstm_layer = LSTM(512, return_state=True)
        
    def call(self, inputs):   
        x = self.embedding_layer(inputs)
        x = self.masking_layer(x)
        lstm1, state_h, state_c = self.lstm_layer(x)
        return state_h, state_c
        

In [28]:
encoder = Encoder()
for x,y in train_dataset.take(1):
    state_h, state_c = encoder(x)
print(tf.shape(state_h))
print(tf.shape(state_c))

KeyboardInterrupt: 

In [ ]:
print(encoder.summary())

## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [ ]:
class Decoder(Model):
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.embedding_layer = Embedding(input_dim=num_unique_german_tokens, output_dim=128, mask_zero=True)
        self.lstm_layer = LSTM(512, return_state=True, return_sequences=True)
        self.dense_layer = Dense(units=num_unique_german_tokens)
        
    def call(self, inputs, hidden_state=None, cell_state=None):  
        x = self.embedding_layer(inputs)
        x, state_h, state_c = self.lstm_layer(x, initial_state=[hidden_state, cell_state])           
        x = self.dense_layer(x)
        return x, state_h, state_c

In [ ]:
decoder = Decoder()
for eng_sentence, ger_sentence in train_dataset.take(1):
    state_h, state_c = encoder(eng_sentence)
    x, state_h, state_c = decoder(ger_sentence, state_h, state_c)
    print(tf.shape(x))

In [ ]:
decoder.summary()

## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [ ]:
optimizer_obj = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
def ger_inputs_outputs(inputs):
    ger_input = inputs[:,:-1]
    ger_output = inputs[:,1:]
    return (ger_input, ger_output)    

In [ ]:
@tf.function
def forward_pass(eng_inputs, ger_inputs, ger_outputs):      
    with tf.GradientTape() as tape:
        state_h, state_c = encoder(eng_inputs)
        ger_outputs_pred, _, _ = decoder(ger_inputs, state_h, state_c)
        loss_value = loss_obj(y_true=ger_outputs, y_pred=ger_outputs_pred)
        encoder_grad, decoder_grad = tape.gradient(loss_value, [encoder.trainable_variables, 
                                                                decoder.trainable_variables])
        return loss_value, encoder_grad, decoder_grad

In [ ]:
epoch_train_loss = []
epoch_val_loss = []

num_epochs = 3

for epoch in range(num_epochs):
    train_loss = tf.keras.metrics.Mean()
    val_loss = tf.keras.metrics.Mean()
    for eng_sentence, ger_sentence in train_dataset:
        ger_inputs, ger_outputs = ger_inputs_outputs(ger_sentence)
        loss_value, encoder_grad, decoder_grad = forward_pass(eng_sentence, ger_inputs, ger_outputs)

        optimizer_obj.apply_gradients(zip(encoder_grad, encoder.trainable_variables))
        optimizer_obj.apply_gradients(zip(decoder_grad, decoder.trainable_variables))
        train_loss(loss_value)
    epoch_train_loss.append(train_loss.result())
    

    for eng_sentence, ger_sentence in val_dataset:
        ger_inputs, ger_outputs = ger_inputs_outputs(ger_sentence)
        state_h, state_c  = encoder(eng_sentence)
        ger_outputs_pred, _, _ = decoder(ger_inputs, state_h, state_c)
        loss = loss_obj(y_true=ger_outputs, y_pred=ger_outputs_pred)
        val_loss(loss)
    epoch_val_loss.append(val_loss.result())
    print(f"epoch {epoch}")

In [ ]:
fig, axes = plt.subplots(1, 2 figsize=(12, 5))

axes[0].set_xlabel("Epochs", fontsize=14)
axes[0].set_ylabel("Loss", fontsize=14)
axes[0].set_title('Training Loss vs Epochs')
axes[0].plot(epoch_train_loss)

axes[1].set_title('Validation Loss vs Epochs')
axes[1].set_ylabel("Loss", fontsize=14)
axes[1].set_xlabel("Epochs", fontsize=14)
axes[1].plot(epoch_val_loss)
plt.show()

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.